In [40]:
import os
import requests
import json
import csv
from datetime import datetime, timedelta, time
import pandas as pd
import requests
from datetime import datetime, timedelta
from datetime import time as dt_time
import time
from tqdm import tqdm

In [41]:
symbols = 'AXP'
# start_date = '2023-01-01T00:00:00Z'
# end_date = '2023-01-31T23:59:59Z'
start_date = datetime(2021, 1, 1, 0,0)
end_date = datetime(2023, 4, 24, 0, 0)

In [42]:
# Fetch stock price data
def fetch_stock_data(ticker, start_date, end_date):
    api_key = os.getenv('APCA_API_KEY_ID')
    secret_key = os.getenv('APCA_API_SECRET_KEY')
    stock_data = {}
    next_page_token = None

    start_date=start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_date=end_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    # Fetch stock price data
    bar_url = f'https://data.alpaca.markets/v2/stocks/{symbols}/bars?start={start_date}&end={end_date}&timeframe=1D'
    headers = {
        'APCA-API-KEY-ID': api_key,
        'APCA-API-SECRET-KEY': secret_key
    }
    bar_resp = requests.get(bar_url, headers=headers)
    bars = bar_resp.json()

    while True:
        bar_url = f'https://data.alpaca.markets/v2/stocks/{ticker}/bars?start={start_date}&end={end_date}&timeframe=1D'
        if next_page_token:
            bar_url+= f'&page_token={next_page_token}'
        headers = {
            'APCA-API-KEY-ID': api_key,
            'APCA-API-SECRET-KEY': secret_key
        }
        bar_resp = requests.get(bar_url, headers=headers)
        bars = bar_resp.json()

        for bar in bars['bars']:
            date = bar['t'][:10]
            stock_data[date] = {
                'Stock Open': bar['o'],
                'Stock Close': bar['c'],
                'Stock High': bar['h'],
                'Stock Low': bar['l'],
                'volume': bar['v'],
                'num_trades': bar['n'],
                'adj_close': bar['vw']
            }

        next_page_token = bars.get('next_page_token')
        if not next_page_token:
            break
    
    return stock_data

stock_data = fetch_stock_data(ticker=symbols, start_date=start_date, end_date=end_date)
print(stock_data)

{'2021-01-04': {'Stock Open': 121.3, 'Stock Close': 118.04, 'Stock High': 121.8, 'Stock Low': 116.85, 'volume': 3472122, 'num_trades': 41872, 'adj_close': 118.388579}, '2021-01-05': {'Stock Open': 118.26, 'Stock Close': 118.67, 'Stock High': 119.28, 'Stock Low': 117.1, 'volume': 2112678, 'num_trades': 28595, 'adj_close': 118.518313}, '2021-01-06': {'Stock Open': 121, 'Stock Close': 123.04, 'Stock High': 124.73, 'Stock Low': 120.81, 'volume': 5861455, 'num_trades': 59274, 'adj_close': 123.36276}, '2021-01-07': {'Stock Open': 124.1, 'Stock Close': 121.66, 'Stock High': 125.6873, 'Stock Low': 117.3501, 'volume': 8696634, 'num_trades': 83972, 'adj_close': 121.131613}, '2021-01-08': {'Stock Open': 122.07, 'Stock Close': 121.78, 'Stock High': 122.195, 'Stock Low': 119.52, 'volume': 2934467, 'num_trades': 36568, 'adj_close': 121.139286}, '2021-01-11': {'Stock Open': 120.42, 'Stock Close': 121.06, 'Stock High': 121.41, 'Stock Low': 119.39, 'volume': 3311225, 'num_trades': 36939, 'adj_close': 1

In [43]:
stock_df = pd.DataFrame.from_dict(stock_data).T

In [44]:
stock_df.index.rename('date', inplace=True)
stock_df.index = stock_df.index.map(pd.to_datetime)
stock_df.head()

,Stock Open,Stock Close,Stock High,Stock Low,volume,num_trades,adj_close
date,,,,,,,
2021-01-04,121.30,118.04,121.8000,116.8500,3472122.0,41872.0,118.388579
2021-01-05,118.26,118.67,119.2800,117.1000,2112678.0,28595.0,118.518313
2021-01-06,121.00,123.04,124.7300,120.8100,5861455.0,59274.0,123.362760
2021-01-07,124.10,121.66,125.6873,117.3501,8696634.0,83972.0,121.131613
2021-01-08,122.07,121.78,122.1950,119.5200,2934467.0,36568.0,121.139286


In [45]:
# Fetch news articles
def paginate_news_requests(api_key, ticker, start_time, end_time, limit=200):
    base_url = "https://www.alphavantage.co/query"
    all_articles = []
    time_from = start_time
    time_to = end_time
    requests_count = 0
    max_daily_requests = 500
    delay_between_requests = 12  # 5 requests per minute

    with tqdm(total=max_daily_requests) as pbar:
        while time_from < time_to and requests_count < max_daily_requests:
            response = requests.get(base_url, params={
                "function": "NEWS_SENTIMENT",
                "tickers": ticker,
                "time_from": time_from.strftime("%Y%m%dT%H%M"),
                "time_to": time_to.strftime("%Y%m%dT%H%M"),
                "limit": limit,
                "apikey": api_key
            })


            requests_count += 1
            pbar.update(1)  # Update the progress bar
            data = response.json()
            articles = data.get('feed', [])

            if not articles:
                print(data)
                break

            all_articles.extend(articles)
            last_article_time = datetime.strptime(articles[-1]['time_published'], "%Y%m%dT%H%M%S")
            time_to = last_article_time - timedelta(seconds=1)

            if time_from < time_to and requests_count < max_daily_requests:
                time.sleep(delay_between_requests)
            else:
                print("Reached end of time range or reached max daily requests")

    return all_articles

# Usage example
api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
print(api_key)
articles = paginate_news_requests(api_key, symbols, start_date, end_date)


UKSF74YC85CMC57Z


  3%|▎         | 13/500 [02:37<1:38:28, 12.13s/it]

{'items': '0', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': []}


In [46]:
articles[-1]

{'title': "'We were terrified': Block co-founder explains how the fintech giant avoided 'death by Amazon'",
 'url': 'https://www.cnbc.com/2022/03/03/how-block-square-avoided-death-by-amazon.html',
 'time_published': '20220304T182803',
 'authors': ['Ryan Browne'],
 'summary': 'In 2014, Amazon launched a competitor to Block. The company was "terrified," according to co-founder Jim McKelvey. But it didn\'t give up.',
 'banner_image': 'https://image.cnbcfm.com/api/v1/image/107023793-1646233560218-gettyimages-472271076-NUP_168700_0001.jpeg?v=1646233640',
 'source': 'CNBC',
 'category_within_source': 'Finance',
 'source_domain': 'www.cnbc.com',
 'topics': [{'topic': 'Technology', 'relevance_score': '0.5'},
  {'topic': 'Finance', 'relevance_score': '0.5'},
  {'topic': 'Financial Markets', 'relevance_score': '0.158519'}],
 'overall_sentiment_score': -0.251747,
 'overall_sentiment_label': 'Somewhat-Bearish',
 'ticker_sentiment': [{'ticker': 'FB',
   'relevance_score': '0.068955',
   'ticker_sen

In [47]:
def save_to_dataframe(articles):
    data = []

    for article in articles:
        data.append({
            "Article Headline": article["title"],
            "Article URL": article["url"],
            "time_published": article["time_published"],
            "Article Text": article["summary"],
            'overall_sentiment_score': article['overall_sentiment_score'],
            'overall_sentiment_label': article['overall_sentiment_label'],
        })

    df = pd.DataFrame(data)
    return df

# Usage example
news_df = save_to_dataframe(articles)

In [48]:
news_df.head()

,Article Headline,Article URL,time_published,Article Text,overall_sentiment_score,overall_sentiment_label
0,JP Morgan Analysis Says Corporate Defined Bene...,https://www.forbes.com/sites/dandoonan/2023/04...,20230424T224258,If companies give pensions a fresh look given ...,0.443382,Bullish
1,Choice Privileges Mastercard Review 2023,https://www.businessinsider.com/personal-finan...,20230424T214200,If you find yourself staying at Choice Hotels ...,0.462092,Bullish
2,LendingClub Appoints Janey Whiteside to its Bo...,https://www.prnewswire.com/news-releases/lendi...,20230424T201000,Former Chief Customer Officer of Walmart and E...,0.268791,Somewhat-Bullish
3,"Wall Street Headlines, International Investing...",https://www.fool.com/investing/2023/04/24/wall...,20230424T194911,"The ""Motley Fool Money"" crew kicks things off ...",0.141750,Neutral
4,Visa Stock Eyes Buy Zone With Earnings Expecte...,https://www.investors.com/news/visa-earnings-e...,20230424T181000,Dow Jones payments processing giant Visa annou...,0.116899,Neutral


In [49]:
# Convert articlePublishedDate to datetime objects
news_df['time_published'] = pd.to_datetime(news_df['time_published'])

# Extract just the date part
news_df['date'] = news_df['time_published'].dt.date

# Define the market close time
market_close_time = dt_time(16, 0, 0)

# Find articles published closest to the market close time
news_df['time_diff'] = news_df['time_published'].apply(lambda x: abs((x - x.replace(hour=market_close_time.hour, minute=market_close_time.minute, second=market_close_time.second)).total_seconds()))
news_df = news_df.sort_values(['date', 'time_diff']).groupby('date', as_index=False).first()

# Drop the 'time_diff' and 'date' columns as they are no longer needed
news_df = news_df.drop(columns=['time_diff', 'time_published'])

news_df.set_index('date',inplace=True)
news_df.index = news_df.index.map(pd.to_datetime)

In [50]:
news_df.head()

,Article Headline,Article URL,Article Text,overall_sentiment_score,overall_sentiment_label
date,,,,,
2022-03-04,'We were terrified': Block co-founder explains...,https://www.cnbc.com/2022/03/03/how-block-squa...,"In 2014, Amazon launched a competitor to Block...",-0.251747,Somewhat-Bearish
2022-03-05,The Best Time to Use Your Airline Miles? Now.,https://www.nytimes.com/2022/03/05/travel/airl...,If you're thinking of traveling and you've got...,0.148399,Neutral
2022-03-06,Blinken says U.S. looking to help supply fight...,https://www.upi.com/Top_News/US/2022/03/06/bli...,"Blinken, speaking to several outlets during a ...",-0.831140,Bearish
2022-03-07,EY joins other big accounting firms exiting Ru...,https://www.nytimes.com/2022/03/07/business/ey...,"The firm said it would sever ties with Russia,...",-0.399577,Bearish
2022-03-08,Crypto exchange boss rejects blanket ban on Ru...,https://www.investorideas.com/news/2022/crypto...,"March 8, 2022 ( Investorideas.com Newswire ) ...",-0.145506,Neutral


In [51]:
final_df = pd.merge(news_df,stock_df, on='date')
final_df.head()

,Article Headline,Article URL,Article Text,overall_sentiment_score,overall_sentiment_label,Stock Open,Stock Close,Stock High,Stock Low,volume,num_trades,adj_close
date,,,,,,,,,,,,
2022-03-04,'We were terrified': Block co-founder explains...,https://www.cnbc.com/2022/03/03/how-block-squa...,"In 2014, Amazon launched a competitor to Block...",-0.251747,Somewhat-Bearish,176.62,172.95,176.820,170.8205,5097583.0,78380.0,172.626008
2022-03-07,EY joins other big accounting firms exiting Ru...,https://www.nytimes.com/2022/03/07/business/ey...,"The firm said it would sever ties with Russia,...",-0.399577,Bearish,170.13,159.13,170.755,159.0800,7423792.0,112507.0,162.584052
2022-03-08,Crypto exchange boss rejects blanket ban on Ru...,https://www.investorideas.com/news/2022/crypto...,"March 8, 2022 ( Investorideas.com Newswire ) ...",-0.145506,Neutral,160.52,160.05,166.290,155.7200,6740476.0,90854.0,160.656545
2022-03-09,"WarnerMedia, Major Tobacco Brands-Here Are The...",https://www.forbes.com/sites/marisadellatto/20...,Here are all the multinationals that have take...,-0.498927,Bearish,168.02,168.65,170.730,166.7100,4551344.0,70611.0,168.802165
2022-03-10,"Food prices are on the rise, but you can still...",https://www.cnbc.com/2022/03/10/how-to-save-mo...,Experts offer their best tips to shield yourse...,0.167264,Somewhat-Bullish,165.59,169.60,170.960,164.8300,3711872.0,64913.0,168.753229


In [52]:
final_df.to_csv(f'data/stock_news_data_{symbols}.csv')